This code runs the experiment for SabreSwap v0.25 and SabreLayout v0.25


In [5]:
from qiskit.transpiler.passes.routing.sabre_swap_v020_lookahead   import SabreSwap   as SabreSwap_v020
from qiskit.transpiler.passes.layout.sabre_layout                 import SabreLayout as SabreLayout_v045
from qiskit.transpiler.passes.layout.sabre_layout_v020            import SabreLayout as SabreLayout_v020
from ulti.circuits import get_circuit_list, sort_circuits_by_depth
from ulti.coupling_maps import file_to_coupling_map
from ulti.pass_managers import generate_pass_managers, transpiled_data
import pandas as pd
import random
random.seed(42)
import os

In [6]:
# Option to keep the layout inital layout the same for faster transpilation
fast_layout = False
num_shots = 5
qc_name = "qvol_10_depth_100"
cp_name = "hex_27"
print(f"Running experiment for the circuits: {qc_name}")
qc_list = get_circuit_list([f'circuits/{qc_name}/'])
qc_list = sort_circuits_by_depth(qc_list)

csv_file_path = f'data/{qc_name}/sabre_v020_look.csv'

directory = f'data/{qc_name}'
if not os.path.exists(directory):
        os.makedirs(directory)

Running experiment for the circuits: qvol_10_depth_100


In [7]:
# Test beam_width from 1
beam_width = list(range(1, 100))
look = 10
all_data_df = pd.DataFrame()
for beam in beam_width:
    print(f"Running experiment for beam {beam}")
    # Experiment for Sabre v0.20 with lookahead heuristic
    # set beam to max integer
    lp_name = "sabre_v020"
    if beam < 10:
        rp_name = f"sabre_v020_look_010_beam_00{beam}"
    elif beam < 100:
        rp_name = f"sabre_v020_look_010_beam_0{beam}"
    else:
        rp_name = f"sabre_v020_look_010_beam_{beam}"

    rp = SabreSwap_v020
    lp = SabreLayout_v020
    cm = file_to_coupling_map(f'coupling_maps/{cp_name}.txt')
    pm = generate_pass_managers(num_shots, rp, lp, cm, beam_width = beam, lookahead = look,
                                fast_layout=fast_layout)

    for qc_idx, qc in enumerate(qc_list):
        data = transpiled_data(qc, pm)
        data['routing pass'] = rp_name
        data['layout pass'] = lp_name
        data['coupling map'] = cp_name
        data['circuit label'] = qc_idx
        data['beam width'] = beam
        data['look ahead'] = look
        
        # Create a DataFrame from the current data and append it to the all_data_df
        current_data_df = pd.DataFrame([data])
        all_data_df = pd.concat([all_data_df, current_data_df])

        # Append the current data to the CSV file
        with open(csv_file_path, 'a') as f:
            current_data_df.to_csv(f, header=f.tell()==0, index=False)

        print(f"    Finished {qc_idx} for experiment {rp_name} and circuit {qc_name}")

    print(f"Finished all for experiment {rp_name} and circuit {qc_name}")
    print(f"Saved to {csv_file_path}")
    print("*" * 50)
    

    


Running experiment for look 1
    Finished 0 for experiment sabre_v020_look_010_beam_001 and circuit qvol_10_depth_100
    Finished 1 for experiment sabre_v020_look_010_beam_001 and circuit qvol_10_depth_100
Finished all for experiment sabre_v020_look_010_beam_001 and circuit qvol_10_depth_100
Saved to data/qvol_10_depth_100/sabre_v020_look.csv
**************************************************
Running experiment for look 2


KeyboardInterrupt: 